# PS4 Coding #

This assignment will have us looking to build a deep convolutional neural network, similar to the architecture of LeNet-5

This network will use the softmax function to make a 10 class image classification on the MNIST data set (the original MNIST and not the fashion_mnist we've been working with thusfar)


But before you get started, please make sure you have the following packages installed
## Packages to install:
1. numpy
2. keras
3. tensorflow
4. matplotlib

For keras and tensorflow, please refer to this link (https://docs.floydhub.com/guides/environments/) to make sure you install versions that are compatible with each other. I would highly recommend getting tensorflow==1.14.1 and the compatible keras version. The exact python version, as long as it's python3+, should not impact your ability to use these two packages.

## Structure of Assigment ##

What's new compared to PS3:
1. **Convolutional Kernels**
3. **Ensemble training**



## Terminology
Please look over the power point under Piazza > Resources > ConvolutionalNetwork.ppt to make sure you understand exactly what I mean when I type the following terms:
1. Applying a kernel/filter
2. Kernel/Filter
3. Max Pool
4. Feature map
5. Convolution

## Network Architecture
![title](./ps4_img/leNet5.png)

You will be implementing variations of LeNet-5 by hand, with flexible kernel shapes. In addition you will be implementing ensemble learning using LeNet-5. 

The static architecture can be seen above:
1. 2 convolutional layers, each followed by a max-pooling layer
2. 3 fully conected layers with an output shape of 10

You will notice that each of the hidden convolutional outputs and max-pooling outputs are ??x?? or ?x? in terms of their dimensions. That is intentional as your first task is to figure out exactly what those are.

1. Kernel 1: 4 x 4, padding = 0, stride = 1
2. Kernel 2: 4 x 4, padding = 0, stride = 1
3. MaxPool1: 2 x 2, padding = 0, stride = 2
4. MaxPool2: 2 x 2, padding = 0, stride = 2

These kernel and maxpool sizes are values to start with. After you get the network working with these kernel and maxpool shapes, you will need to adjust it so it can take any valid kernel and any valid max pool shape. 

Here, we define valid as **out_shape is an integer greater than 1**, where $$out\_shape = \frac{input\_shape + 2 * padding - kernel\_shape}{stride} + 1$$

Static Variables:
1. Number of layers
2. Output shape

Flexible variables:
1. Kernel shapes
2. MaxPool filter shapes
3. Number of kernels per conv layer
4. Number of nodes per FC layer

## Assignment Grading and Procedure Recommendation ##
This assignment overall has ??? points for all the methods you have to implement. Imposed on this total are the following percentages:

1. If you correctly implement all methods, and you can correctly apply one kernel per conv layer, you will earn 80% of the points
2. Correct implementation of multiple kernels applied at each layer will earn you an additional 10%
3. Correct implementation of ensemble learning will earn you an additional 5%
4. Experimentation on parameters will earn you the last 5% of the points. See the bottom of this document for details

For instance, if you correctly implement all methods but do not have multiple kernels, nor ensemble training you will recieve (87 x 0.75) out of the possible 87. 

If you accomplish the situation above but also correctly add drop out and Ensemble training, you will then recieve (87 * 0.95) out of the possible 87.

Here is how I recommend going about this assignment:
1. Implement the network with batch training, weight decay, bias terms, and one kernel applied to each conv layer
2. Add multiple kernels per conv layer functionality
3. Add different kernel shape functionality
4. Implement ensemble learning
5. Experimentation

### A note on "different kernel shape": 
For a convolutional network, all kernels applied at the same layer will be the **same shape**, when I say that your network should be able to handle differnt kernel shapes, that means if you change the kernel shape at a given layer, **all kernels applied on that layer will adopt the new shape**.

For instance, Kernel1 begins as a 4x4 kernel. This means if I wanted to apply multiple Kernel1's to the input layer, then I will apply multiple **4x4 kernels** (they are all the same shape). If I change my network such that Kernel1 is now a **6x6 kernel** that means ALL applications of kernel1 will now be 6x6.

### Data Format

You will notice that this assignment has very little headers and comments. I am leaving it up to you to decide exactly what info you need to incorporate for each function as a parameter, and the functionality and output of each that function. Feel free to use the previous problem sets as models for how to model your code. I recommend you continue to format your data in terms of N x M

1. N = number of features
2. M = number of data points

## Loops during multi-kernel convolution

In order to not get you guys bogged down on dealing with 3D and 4D matrix multiplication, I will say the following:

The application of a single kernel should not impose any loops (straight matrix multiplication).

However, when you reach the stage of applying multiple kernels to a single layer, I would recommend you simply loop through all kernel matrices for that layer and apply them one at a time.

This means that if your data begins as a NxM (2D matrix), then each kernel application will produce a (N1 x M) 2D matrix, where N1 = the flattened feature map of the kernel application. **These (N1 x M) matrices can be kept separately, rather than combining them into one 3D matrix. **

## Data management

We will be using four data sets for this problem set. 
1. MNIST (the most popular computer vision data set)
2. Dummy data (for testing purposes)

**Graded Exercise (15 points total, 3 points each) - implement the following functions for data parsing:** 

In [18]:
from keras.datasets import cifar10
import tensorflow as tf
import keras
from matplotlib import pyplot
import numpy as np
####BEGIN CODE HERE####
def gen_dummy():
    '''
    dummy data is exceptionally useful to test whether or not your network behaves as expected.
    For dummy data, you should generate a few (<= 5) input/output pairs that you can use to test
    your forward and backward propagation algorithms
    
    output: 
    dummy_x = a NxM np matrix, both dimensions of your choosing of very simple data
    dummy_y = a (M, ) np array with the corresponding labels
    '''
    dummy_x = []
    dummy_y = []
    ####BEGIN CODE HERE####
    
    N = 28 * 28
    M = 5
    
    dummy_x = np.ones((N, M))
    dummy_y = np.ones(M)
    ####END CODE HERE####

    return dummy_x, dummy_y


def load_mnist():
    ''' 
    look up how to load the mnist data set via keras
    '''
    mnist = keras.datasets.mnist
    (mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = mnist.load_data()
    return mnist_train_images, mnist_train_labels, mnist_test_images, mnist_test_labels


def flatten_normalize(images):
    '''
    convert the image from a N1xN1xM to NxM format where N1 = square_root(N) and normalize
    '''
    images = np.reshape(images, (-1, images.shape[-1]))
    images = np.divide(images, np.max(images, 1))
    return images


def subset_mnist_training():
    '''
    Return 100 training samples from each of the 10 classes, 1000 samples all together
    '''
    mnist_train_images, mnist_train_labels, mnist_test_images, mnist_test_labels = load_mnist()
    
    images = []
    labels = []
    
    for i in np.unique(mnist_train_labels):
        train = mnist_train_images[mnist_train_labels == i]
        train = train[np.random.choice(train.shape[0], 100, False)]
        images.extend(train)
        labels.extend([i for k in range(100)])
        
    return images, labels


def subset_mnist_testing():
    '''
    Return 20 training samples from each of the 10 classes, 200 samples all together
    '''
    mnist_train_images, mnist_train_labels, mnist_test_images, mnist_test_labels = load_mnist()
    
    images = []
    labels = []
    
    for i in np.unique(mnist_test_labels):
        test = mnist_test_images[mnist_test_labels == i]
        test = test[np.random.choice(test.shape[0], 20, False)]
        imgs.extend(test)
        labels.extend([i for k in range(20)])
        
    return images, labels
    
####END CODE HERE



**Graded exercise (28 points total, 4 points each): Complete the following helper functions**

You will notice that there are no parameters, that is up to you to determine what each function needs. You will also notice there isn't much explanation as to what each function does. That is because you should determine what each function takes as parameters and what they return. The only thing I ask you not to change is the function name


In [30]:
####BEGIN CODE HERE####
def log_cost(label, prediction):
    '''
    computes the log cost of the current predictions using the labels (same as PS3)
    '''
    cost = 0
    for i in range(label.shape[0]):
        cost += -np.log(prediction[int(label[i]), i])
    return cost


def softmax(x):
    '''
    computes the softmax of the input (same as PS3)
    '''
    ret = np.zeros(x.shape)
    for col in range(x.shape[1]):
        total = np.sum(np.exp(x[:, col]))
        for row in range(x.shape[0]):
            ret[row,col] = np.exp(x[row,col])/total
    return ret


def ReLU(x):
    '''
    computes the ReLU of the input (same as PS3)
    '''
    return np.maximum(0, x)


def ReLU_prime(x):
    ''''
    computes the ReLU' of the input (same as PS3)
    
    '''
    ret = np.zeros(z.shape)
    for i in range(z.shape[0]):
        for j in range(z.shape[1]):
            if z[i][j] > 0:
                ret[i][j] = 1
            else:
                ret[i][j] = 0
    return ret


def kernel_to_matrix(kernal: np.ndarray, input_size):
    '''
    converts a kernel to its matrix form
    '''
    kernal_size = kernal.shape[0]
    output_size = input_size - kernal_size + 1

    kernal = kernal.flatten()
    to_right = input_size - kernal_size
    ret = np.zeros((output_size * output_size, input_size * input_size))
    t0 = 0
    for i in range(output_size * output_size):
        t1 = i
        for j in range(kernal.shape[0]):
            ret[i, j + t0 + t1] = kernal[j]
            if (j + 1) % kernal_size == 0:
                t1 += to_right
        if (i + 1) % output_size == 0:
            t0 += (kernal_size - 1)
    return ret


def max_pool(X):
    '''
    applies max-pooling to an input image
    '''
    size = int(np.sqrt(X.shape[0]))
    
    output_size = size // 2
    ret = np.empty((output_size, output_size, X.shape[-2], X.shape[-1]))
    for i in range(X.shape[1]):
        for j in range(X.shape[2]):
            t1 = np.reshape(X[:, i, j], (size, size))
            for x in range(output_size):
                for y in range(output_size):
                    ret[x, y, i, j] = np.max(t1[x * 2: (x + 1) * 2, y * 2: (y + 1) * 2])
                    
    return np.reshape(ret, (output_size * output_size, X.shape[-2], X.shape[-1]))


def max_pool_backwards(delta_el, y):
    '''
    takes the output of a maxpool, and projects back to the original shape.
    see PPT slides on convolutional backprop if you have no idea what I'm talking about.
    '''
    small_size = int(np.sqrt(delta_el.shape[0]))
    big_size = small_size * 2 + 1
    origin = np.reshape(delta_el, (small_size, small_size, delta_el.shape[-2], delta_el.shape[-1]))
    y = np.reshape(z, (big_size, big_size, z.shape[-2], z.shape[-1]))
    ret = np.zeros((big_size, big_size, delta_el.shape[-2], delta_el.shape[-1]))
    for b in range(origin.shape[-2]):
        for layer in range(origin.shape[-1]):
            for i in range(small_size):
                for j in range(small_size):
                    index = np.unravel_index(np.argmax(z[i * 2: (i + 1) * 2, j * 2: (j + 1) * 2, b, layer].flatten()), (2, 2))
                    ret[i * 2 + index[0], j * 2 + index[1], b, layer] = origin[i, j, b, layer]
    return ret.reshape((-1, ret.shape[-2], ret.shape[-1]))


####END CODE HERE####


**Graded exercise (6 points total, 2 points per function) - Complete the initialization functions**

In [31]:
####BEGIN CODE HERE####
def kernel_initialization(w1, w2):
    '''
    returns a kernel with specified height and width. 
    The values of the kernel should be initialized using the same formula as the
    He_initialize_weight() function
    '''
    ret = np.random.rand(w1, w2) * np.sqrt(2 / max(w1, w2))
    return ret


def He_initialize_weight(w1, w2):
    '''
    (same as PS3)
    returns a weight matrix with the passed in dimensions
    '''
    ret = np.random.rand(w1, w2) * np.sqrt(2 / max(w1, w2))
    return ret


def bias_initialization(w1):
    ''' (same as PS3)
    returns a bias matrix of the passed in dimensions
    '''
    ret = np.random.rand(w1) * np.sqrt(2 / w1)
    return ret
####END CODE HERE####

## Forward and BackProp functions

![title](./ps4_img/backprop.png)

**Graded Exercise (28 points total, 4 points each) - complete the following functions:**
1. predict()
2. delta_Last()
3. delta_el()
4. dW()
5. db()
6. weight_update()
7. bias_update()

For full credit for predict, you will need to incorporate the bias terms correctly, and for weight_update, you will need to correctly utilize the weight_decay parameter

You may find the image above helpful when implementing the backpropagation methods

In [32]:
####BEGIN CODE HERE####
def predict(x, weights, bias, activations):
    '''
    minimum output: the predictions made by the network
    
    You are free to return more things from this function if you see fit
    
    hint: you will need to return all intermediate computations, not just the output
    To figure out what you need to return, look at what intermediate results you need
    to compute backpropagation
    
    you can return more than one variable with the following syntax:
    
    return var1, var2, ..., varN
    '''
    out = x
    z = [x]
    a = [x]
    for i in range(len(activations)):
        if activations[i] == "max" and isinstance(weights[i], list):
            if len(out.shape) <= 2:
                out = out[..., np.newaxis]
            layer_num = 1 if len(out.shape) <= 2 else out.shape[2]
            t = np.empty((weights[i][0].shape[0], out.shape[1], len(weights[i]) * layer_num))
            for j in range(len(weights[i])):
                for k in range(layer_num):
                    t[:, :, j * layer_num + k] = np.matmul(weights[i][j], out[:, :, k])
            out = t
            if activations[i + 1] == 'relu':
                z.append(np.transpose(out, (0, 2, 1)).reshape((-1, out.shape[-2])))
            else:
                z.append(out)
            out = max_pool(out)
            a.append(out)
        elif activations[i] == 'relu':
            if len(out.shape) > 2:
                out = np.transpose(out, (0, 2, 1))
                out = out.reshape((-1, out.shape[-1]))
                a[-1] = out
            out = np.add(np.matmul(weights[i].T, out), bias[i][..., np.newaxis])
            z.append(out)
            out = ReLU(out)
            a.append(out)
        elif activations[i] == "softmax":
            out = softmax(out)
            a.append(out)
        else:
            raise Exception(" Wrong activation")
    return out, z, a


def delta_Last(predictions, labels):
    '''
    task: computer error term for ONLY output layer
    '''
    return predictions - labels


def delta_el(weight_d_plus_1, delta_d_plus_1, input_d):
    '''
    task: compute error term for any hidden layer
    '''
    ####BEGIN CODE HERE ####
    return np.matmul(weight_d_plus_1, delta_d_plus_1) * ReLU_prime(input_d)
    ####END CODE HERE####


def dW(delta_d, a_d_minus_1):
    '''
    task: compute gradient for any weight matrix
    '''
    return np.matmul(delta_d, a_d_minus_1.T)


def db(delta_d):
    '''
    task: compute gradient for any bias term
    '''
    return np.sum(delta_d, axis=1, keepdims=True)


def weight_upate(weights, gradients, learning_rate):
    '''

    task: udpate each of the weight matrices, and return them in a variable, name of  your choosing
    '''
    for i in range(len(weights)):
        weights[i] -= learning_rate * gradients[i]
    return weights


def bias_update(bias, bias_grad, learning_rate):
    '''
    task : update each of the bias terms, return them in a variable, name of  your choosing
    '''
    for i in range(len(bias)):
        bias[i] -= learning_rate * bias_grad[i]
    return bias



    

## TRAINING

**Graded Exercise (10 points): Complete the training function below**

In [33]:
def train(x, y, weights, bias, activations, epoch, lr):
    '''
    IN ADDITION: please have the train function output a graph of the cost using matplotlib.pyplot. 
    If everything works correctly the cost should be monotonically decreasing
    '''
    ####BEGIN CODE HERE####

    num_samples = x.shape[1]

    for i in range(epoch):
        print('--------epoch', i, "--------------")
        out, z, a = predict(x, weights, bias, activations)
        # pred_y = decision(out)
        # acc = np.sum(pred_y == y) / len(y)
        cost = log_cost(y, out)
        print("epoch: ", i, " cost: ", cost)
        # backward
        delta_l = delta_Last(out, y)
        d_W = []
        d_b = []
        for j in range(2, len(activations) - 2):
            if activations[-j] == "relu":
                d_W_t = dW(delta_l, a[-j - 1]).T / num_samples
                d_b_t = db(delta_l) / num_samples
                d_W.append(d_W_t)
                d_b.append(d_b_t)
                if activations[-j - 1] != "relu":
                    delta_l = delta_el(weights[-j + 1], delta_l, a[-j - 1])
                else:
                    delta_l = delta_el(weights[-j + 1], delta_l, z[-j - 1])
            elif activations[-j] == "max":
                layer = 1
                pre_z = z[-j + 1]
                for k in range(len(weights) - j + 2):
                    layer = layer * len(weights[k])
                if activations[-j + 1] != "max":
                    delta_l = delta_l.reshape((-1, layer, delta_l.shape[-1]))
                    pre_z = pre_z.reshape((-1, layer, delta_l.shape[-1]))
                delta_l = max_pool_backwards(delta_l, pre_z)
                d_W_t = []
                d_b_t = []
                for k in range(layer):
                    d_W_t.append(dW(delta_l[:, k, :], a[-j - 1][:, :, k // len(weights[-j + 1])]).T / num_samples)
                    d_b_t.append(db(delta_l[:, k, :]) / num_samples)
                d_W.append(d_W_t)
                d_b.append(d_b_t)
                if j == len(activations):
                    continue
                t_delta_l = []
                for k in range(layer // len(weights[-j + 1])):
                    t_delta_l.append(delta_el(weights[-j][k], delta_l[:, k * len(weights[-j+1]), :], z[-j - 1]))
                    t_delta_l.append(delta_el(weights[-j][k], delta_l[:, k * len(weights[-j+1]) + 1, :], z[-j - 1]))

            else:
                raise Exception()
            weights = weight_upate(weights, d_W, lr)
            bias = bias_update(bias, d_b, lr)
    return weights, bias
    ####END CODE HERE####


In [34]:
'''
    In the section below is where you should setup all your weights, bias, parameters, and input/labels 
    
    Then pass all the relevant parameters to train and run and debug it
    
    Remember: no aspect of the network size should be hard coded except the output layer (which should always
    have three nodes), they should adjustable via variables like the ones provided. 
    
    The size of your input layer will be dictated by which data set you are using. Once again, that parameter should be
    in terms of the variables, and not a hard coded size

'''
output_nodes = 10
####BEGIN CODE HERE####
bias_0 = bias_initialization(2)
bias_1 = bias_initialization(2)
bias_2 = bias_initialization(10)
bias = [bias_0, bias_1, bias_2]

weights_0 = []
ratio_0 = 2
for i in range(ratio_0):
    t = kernel_initialization(4, 4)
    weights_0.append(kernel_to_matrix(t, 28))
# weights_0 = np.asarray(weights_0)

weights_1 = []
ratio_1 = 2
for i in range(ratio_1):
    t = kernel_initialization(4, 4)
    weights_1.append(kernel_to_matrix(t, 12))
# weights_1 = np.asarray(weights_1)

weights_2 = He_initialize_weight(64, 10)
weights = [weights_0, weights_1, weights_2]

activations = ['max', 'max', 'relu', 'softmax']
x, y = gen_dummy()
train(x, y, weights, bias, activations, 10, 0.01)

####END CODE HERE####

--------epoch 0 --------------
epoch:  0  cost:  34.89302805859221
--------epoch 1 --------------
epoch:  1  cost:  34.89302805859221
--------epoch 2 --------------
epoch:  2  cost:  34.89302805859221
--------epoch 3 --------------
epoch:  3  cost:  34.89302805859221
--------epoch 4 --------------
epoch:  4  cost:  34.89302805859221
--------epoch 5 --------------
epoch:  5  cost:  34.89302805859221
--------epoch 6 --------------
epoch:  6  cost:  34.89302805859221
--------epoch 7 --------------
epoch:  7  cost:  34.89302805859221
--------epoch 8 --------------
epoch:  8  cost:  34.89302805859221
--------epoch 9 --------------
epoch:  9  cost:  34.89302805859221


([[array([[0.56869434, 0.61126912, 0.28945998, ..., 0.        , 0.        ,
           0.        ],
          [0.        , 0.56869434, 0.61126912, ..., 0.        , 0.        ,
           0.        ],
          [0.        , 0.        , 0.56869434, ..., 0.        , 0.        ,
           0.        ],
          ...,
          [0.        , 0.        , 0.        , ..., 0.26291215, 0.        ,
           0.        ],
          [0.        , 0.        , 0.        , ..., 0.15822765, 0.26291215,
           0.        ],
          [0.        , 0.        , 0.        , ..., 0.153822  , 0.15822765,
           0.26291215]]),
   array([[0.08666708, 0.26877927, 0.69540879, ..., 0.        , 0.        ,
           0.        ],
          [0.        , 0.08666708, 0.26877927, ..., 0.        , 0.        ,
           0.        ],
          [0.        , 0.        , 0.08666708, ..., 0.        , 0.        ,
           0.        ],
          ...,
          [0.        , 0.        , 0.        , ..., 0.57373829, 0.  

## Testing ##

There are two steps to the testing process: first, we need to take the output of our network and make a decision, either class 0, class 1, ..., or class 10, and second we need to measure how well our network performs on the testing set.

Remember that our network outputs 10 probability values, all between 0 and 1, and we need turn this vector of three elements into a single output: the predicted class of the data point by the network. Simply pick the index of the largest probability value as the decision. 

**Graded Exercise(5 points total) - implement decision (1 point) and test(4 points)**

In [7]:
####BEGIN CODE HERE####
def decision(prediction):
    '''
    input: a (10, M) matrix where each column is the softmax output for data point i, 0 <= i <= M
    
    output: a (M, ) np array where each element corresponds to the highest probability class from prediction
    '''
    ret = np.argmax(prediction, axis=1)
    return ret


def test(x, y, weights, bias):
    '''
    output: the accuracy of your model on the unseen x and y pairs
    '''

    output, _, _ = predict(x, weights, bias)
    output = decision(output)
    accuracy = np.sum(output == y) / len(y)
    return accuracy

####END CODE HERE####

In [8]:
'''
Here is some space for you to call and test your test/decision functions
'''
####BEGIN CODE HERE####

####END CODE HERE####

'\nHere is some space for you to call and test your test/decision functions\n'

## Ensemble learning: 

This is a fairly common technique to try and promote regularization in a system. Simply put, you train multiple networks (either of the same or different sizes/shapes), on the same data. 

Then, you pass each of the networks a testing set, and initiate a "vote". 

The voting procedure is: for each testing point, the class with majority vote wins.

If none of your models agree on a class, then you just randomly pick from their decisions. Tie breakers are also determined randomly.

To recieve full credit for ensemble learning, simply train a minimum of 5 unique networks, and correctly code a voting process

In [9]:
####BEGIN CODE HERE####
''' 
ensemble training and testing space
'''

####END CODE HERE####

' \nensemble training and testing space\n'

## Experimentation explanation:

What are you experimenting on? Any aspect of your network that is not learned by SGD, and is not the number of layers of the network, is free for experimentation. Learning rate, number of training samples, epochs, nodes per layer etc etc are a few examples.

All I ask in this section, is that you try to maximize your performance on the fashion_mnist data set and mnist dataset on at least 60 testing samples, and keep track of what you found below. The write up can be as simple as: "The best I got with mnist was (accuracy) with these parameters: (list of parameters)". But ideally you would talk a bit more about the trends, such as "I noticed that if i kept decreasing my learning rate, the performance on X dataset would improve until a certain point, then would become worse". 

There are a lot of ways to get full credit on this experimentation portion, as there is no dedicated format I am requesting. There is no benchmark. There is no "you must get 100% accuracy". I simply ask you to see how well your models can do with the restrictions in place.

Best of luck, 
Ryan

Experimentation notes:

    This cell has been left in text format for you to freely edit and keep track of your experimentations.